<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RS_gr1/blob/main/Marfida%20Alexander/A_Marfida_stajirovka_profpatolog_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Стажировка. Профпатолог (INSPECTRUM CLINIC)
- Подготовка датасетов

## Загрузка датафрейма

In [ ]:
# Подключение библиотек

# Работа с массивами данных
import numpy as np 

# Работа с табличными данными
import pandas as pd

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential, Model

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation, Input, concatenate
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D

# Оптимизаторы
from tensorflow.keras.optimizers import Adam, Adadelta, SGD, Adagrad, RMSprop

# Токенизатор для преобразования текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Масштабирование данных
from sklearn.preprocessing import StandardScaler

# Загрузка датасетов из облака google
import gdown

# Регулярные выражения
import re

# Отрисовка графиков
import matplotlib.pyplot as plt
plt.style.use('dark_background')

# Создание статистических графиков
import seaborn as sns

# Табличные данные
from fastai.tabular import *

# Метрики для расчета ошибок
from sklearn.metrics import mean_squared_error, mean_absolute_error

%matplotlib inline

In [ ]:
# Монтирование Goodle-диска
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_backup = pd.read_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICdata_I(71).xlsx')

In [ ]:
# Создание рабочей копии датафрейма
df = df_backup.copy()

## Служебные функции

In [ ]:
# Глобальные переменные
EMPTY_FIELDS = 'Не определено'

### Функции подсчёта заполненности колонок

In [ ]:
# Аргументы - датафрейм и список наименований столбцов
# Исключения: NaN
#
def count_columns_fill_not_na(df, arg_list):
  count_list = 0
  print('Текущая структура датафрейма: ', df.shape, '\n')
  for col_name in arg_list:
    count_list += 1
    print(f'*** {col_name}: ***') 
    print('Количество непустых элементов: ', len(df[~df[col_name].isna()]) )
    print('Количество уникальных элементов: ', len(df.loc[~df[col_name].isnull(), col_name].unique()), '\n')
  print('Количество столбцов: ', count_list)

In [ ]:
# Аргументы - датафрейм и список наименований столбцов
# Исключения: EMPTY_FIELDS ("Не определено")
#
def count_columns_fill(df, arg_list):
  count_list = 0
  print('Текущая структура датафрейма: ', df.shape, '\n')
  for col_name in arg_list:
    count_list += 1
    print(f'*** {col_name}: ***') 
    print('Количество непустых элементов: ', len(df[df[col_name] != EMPTY_FIELDS]) )
    print('Количество уникальных элементов: ', len(df.loc[df[col_name] != EMPTY_FIELDS, col_name].unique()), '\n')
  print('Количество столбцов: ', count_list)

### Функция, возвращающая количество значащих записей и уникальных элементов в столбце

In [ ]:
# Аргументы - датафрейм и строка наименования столбца
# return: count_fills, count_unique
# 
def count_fill_and_unique(df, col_name):
  return len(df[df[col_name] != EMPTY_FIELDS]), len(df.loc[df[col_name] != EMPTY_FIELDS, col_name].unique())


### Функция просмотра содержания ненулевых уникальных значений полей в выбранных колонках

In [ ]:
def prn_unique_txt_col(df, *arg, sort_asc = False):
  for colname in arg:
    column_list = list(df.loc[~df[colname].isnull(), colname].unique())
    count_list = len(column_list)
    print(f'*** {colname}({count_list} уникальных, вкючая не заданные): ***')
    to_join = sorted(column_list) if sort_asc else column_list
    print('\n'.join(to_join))
    print()

### Датафрейм исходных данных для сборок датасетов обучения

In [ ]:
dfc_data = {
    'name':[],
    'index':[],
    'fill_cnt':[],
    'unique_cnt':[],
    'data_type':[],
    'train_sets':[]
}
dfc = pd.DataFrame(dfc_data)

In [ ]:
for col in df:
  dfc.loc[len(dfc.index)] = [
    col,
    df.columns.get_loc(col),
    len(df[df[col] != EMPTY_FIELDS]),
    len(df.loc[df[col] != EMPTY_FIELDS, col].unique()),
    df[col].dtype,
    0
  ]
dfc

,name,index,fill_cnt,unique_cnt,data_type,train_sets
0,КлиентДатаРождения,0,1922,1794,datetime64[ns],0
1,КлиентПол,1,1922,2,object,0
2,Контрагент,2,1922,177,object,0
3,Профессия,3,1922,623,object,0
4,ЗаключениеМК,4,1515,3,object,0
...,...,...,...,...,...,...
66,Офтальмология2_ПрофнепригодностьВременнаяПредс...,66,48,2,object,0
67,Офтальмология2_ПрофнепригодностьВременнаяПредс...,67,45,2,object,0
68,Офтальмология2_Диагноз_ЗначениеПредставление,68,51,17,object,0
69,Терапия2_ПрофнепригодностьВременнаяПредставление2,69,73,3,object,0


In [ ]:
count_columns_fill(df, ['ПроцедурнаяМедсестра1_МКБ101'])

Текущая структура датафрейма:  (1922, 71) 

*** ПроцедурнаяМедсестра1_МКБ101: ***
Количество непустых элементов:  75
Количество уникальных элементов:  2 

Количество столбцов:  1


In [ ]:
prn_unique_txt_col(df, 'ПроцедурнаяМедсестра1_МКБ101')

*** ПроцедурнаяМедсестра1_МКБ101(3 уникальных, вкючая "Не определено"): ***
Не определено
Z00.0
H52.1



In [ ]:
df

,КлиентДатаРождения,КлиентПол,Контрагент,Профессия,ЗаключениеМК,ВредныеФакторы,ПсихиатрияНаркология1_МКБ101,ПсихиатрияНаркология1_ПрофнепригодностьВременнаяПредставление1,ПсихиатрияНаркология1_ГоденНегоден_ЗначениеПредставление,ПсихиатрияНаркология1_Диагноз_ЗначениеПредставление,...,Дерматовенерология2_Диагноз_ЗначениеПредставление,Терапия2_Диагноз_ЗначениеПредставление,Хирургия1_ПрофнепригодностьВременнаяПредставление2,Хирургия2_ПрофнепригодностьВременнаяПредставление2,Офтальмология2_МКБ101,Офтальмология2_ПрофнепригодностьВременнаяПредставление1,Офтальмология2_ПрофнепригодностьВременнаяПредставление2,Офтальмология2_Диагноз_ЗначениеПредставление,Терапия2_ПрофнепригодностьВременнаяПредставление2,Терапия1_Диагноз_ЗначениеПредставление
0,1976-10-24,Женский,23c173a7-30d9-11e7-80d8-708bcd7f83cd,Ведущий специалист по организации обучения,Годен,11.4,Z00.0,Не определено,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено
1,1987-11-18,Мужской,a6d559b0-7835-11eb-80e8-0cc47aab8067,Ведущий специалист,Годен,4.2.5,Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено
2,1985-09-08,Мужской,a6d559b0-7835-11eb-80e8-0cc47aab8067,Главный менеджер,Годен,4.2.5,Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено
3,1988-04-25,Мужской,a6d559b0-7835-11eb-80e8-0cc47aab8067,Главный специалист,Годен,11.1,Не определено,Не определено,Не определено,Не определено,...,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено
4,1984-04-17,Мужской,a6d559b0-7835-11eb-80e8-0cc47aab8067,Старший специалист,Годен,11.1,Не определено,Не определено,Не определено,Не определено,...,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1917,1972-11-12,Мужской,a9f70e4c-707f-11ed-8109-0cc47aab8067,Директор по строительству,Не определено,6.1,Не определено,Не определено,Не определено,Не определено,...,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено
1918,1986-07-09,Мужской,a9f70e4c-707f-11ed-8109-0cc47aab8067,Начальник производственно-технического отдела,Не определено,6.1,Не определено,Не определено,Не определено,Не определено,...,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено
1919,1994-05-12,Мужской,e689a382-17b2-11e9-80ce-0cc47aab8067,Помощник кладовщика,Не определено,"5.1,23",Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,[Z00.0] Общий медицинский осмотр_x000D_\n
1920,1972-04-17,Женский,21f2a93d-68f9-11e6-82dc-d897ba0a5c60,Повар,Не определено,23,Z00.0,Годен,Годен,[Z00.0] Общий медицинский осмотр_x000D_\n,...,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,Не определено,[Z00.0] Общий медицинский осмотр_x000D_\n


In [ ]:
pip install thefuzz[speedup]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.4 MB/s eta 0:00:00


In [ ]:
from thefuzz import fuzz
from thefuzz import process

In [ ]:
#fuzz.ratio("	[Z00.0] Общий медицинский осмотр_x000D_", "	[Z00.0]")
fuzz.ratio("4.2.5,1.29.2,4.1,4.7,4.4,6.2,11.1", "4.2.5,11.1,11.4,13")

54

In [ ]:
#fuzz.partial_ratio("	[Z00.0] Общий медицинский осмотр_x000D_", "	[Z00.0]")
fuzz.partial_ratio("4.2.5,1.29.2,4.1,4.7,4.4,6.2,11.1", "4.2.5,11.1,11.4,13")

61

In [ ]:
#fuzz.token_sort_ratio("[Z00.0] Общий медицинский осмотр_x000D_", "Общий медицинский осмотр_x000D_")
fuzz.token_sort_ratio("4.2.5,1.29.2,4.1,4.7,4.4,6.2,11.1", "4.2.5,11.1,11.4,13")

63

In [ ]:
pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import Levenshtein

In [ ]:
Levenshtein.distance(
    '4.2.5,1.29.2,4.1,4.7,4.4,6.2,11.1',
    '4.2.5,11.1,11.4,13'
)

19

----

In [ ]:
df.shape

(1922, 71)

#### Заменим все NaN на 'Не определено'

In [ ]:
df.describe(include = "all")

In [ ]:
df.isna().sum().sum()

85613

In [ ]:
df = df.fillna('Не определено')

In [ ]:
df.isna().sum().sum()

0

### Подготовка глобальных данных и разбивка на блоки

In [ ]:
df.shape

(1922, 71)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1922 entries, 0 to 1921
Data columns (total 71 columns):
 #   Column                                                          Non-Null Count  Dtype         
---  ------                                                          --------------  -----         
 0   КлиентДатаРождения                                              1922 non-null   datetime64[ns]
 1   КлиентПол                                                       1922 non-null   object        
 2   Контрагент                                                      1922 non-null   object        
 3   Профессия                                                       1922 non-null   object        
 4   ЗаключениеМК                                                    1922 non-null   object        
 5   ВредныеФакторы                                                  1922 non-null   object        
 6   ПсихиатрияНаркология1_МКБ101                                    1922 non-null   object  

In [ ]:
count_columns_fill(df, list(df.columns))

Текущая структура датафрейма:  (1922, 71) 

*** КлиентДатаРождения: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  1794 

*** КлиентПол: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  2 

*** Контрагент: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  177 

*** Профессия: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  623 

*** ЗаключениеМК: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  4 

*** ВредныеФакторы: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  512 

*** ПсихиатрияНаркология1_МКБ101: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  10 

*** ПсихиатрияНаркология1_ПрофнепригодностьВременнаяПредставление1: ***
Количество непустых элементов:  1922
Количество уникальных элементов:  5 

*** ПсихиатрияНаркология1_ГоденНегоден_ЗначениеПредставление: ***
Количество непустых элементов:  1922
К

In [ ]:
dnm = 'df'
cmd = dnm + "['ПроцедурнаяМедсестра1_МКБ101']"
eval(cmd)

0       Не определено
1       Не определено
2       Не определено
3       Не определено
4       Не определено
            ...      
1917    Не определено
1918    Не определено
1919    Не определено
1920    Не определено
1921    Не определено
Name: ПроцедурнаяМедсестра1_МКБ101, Length: 1922, dtype: object

### Сборка словаря со сводными данными для формирования датасетов

In [ ]:
df.to_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICdata_I(71)_1.xlsx', index=False)